- to check convergence of BB and CC

# recsys2021 best paper
- paper: https://dl.acm.org/doi/pdf/10.1145/3460231.3474273
- code: https://github.com/hasteck/Higher_Recsys_2021/blob/main/Higher_RecSys_2021.ipynb

In [1]:
import os
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from numba import jit
from copy import deepcopy
from scipy import sparse, linalg

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = "/kaggle/input/h-and-m-personalized-fashion-recommendations/"
df = pd.read_csv(DATA_DIR + "transactions_train.csv")
#article_df = pd.read_csv(DATA_DIR + "articles.csv")
#customer_df = pd.read_csv(DATA_DIR + "customers.csv")
sub_df = pd.read_csv(DATA_DIR + "sample_submission.csv")

In [3]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

def apk(actual, predicted, k=12):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]
        
    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## prepare data

In [4]:
start_date = '2020-09-18'
val_start_date = '2020-09-21' #9/16

df = df[["t_dat", "customer_id", "article_id"]]

# sample check for small memoery usage
df = df.query(f"t_dat >= '{start_date}'").reset_index(drop=True)
train_data = df.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)

In [5]:
unique_uid = df["customer_id"].unique()
unique_sid = df['article_id'].unique()

show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

n_users = max(list(profile2id.values())) + 1
n_items = max(list(show2id.values())) + 1
print(n_users, n_items)

del df, unique_uid, unique_sid

51023 16047


In [6]:
def numerize(tp):
    uid = map(lambda x: profile2id[x], tp['customer_id'])
    sid = map(lambda x: show2id[x], tp['article_id'])
    out = pd.DataFrame(data={'uid': list(uid), 'sid': list(sid)}, columns=['uid', 'sid'])
    return out.sort_values("uid").reset_index(drop=True)

In [7]:
train_data = numerize(train_data)
print(train_data.shape)

(107569, 2)


# training

In [8]:
# function for training
### functions to create the feature-pairs
@jit
def create_list_feature_pairs(XtX, threshold):
    AA= np.triu(np.abs(XtX))
    AA[np.diag_indices(AA.shape[0])]=0.0
    ii_pairs = np.where((AA>threshold)==True)
    return ii_pairs

@jit
def create_matrix_Z(ii_pairs, X):
    MM = np.zeros( (len(ii_pairs[0]), X.shape[1]), dtype=np.float)
    MM[np.arange(MM.shape[0]), ii_pairs[0]]=1.0
    MM[np.arange(MM.shape[0]), ii_pairs[1]]=1.0
    CCmask = 1.0-MM    # see Eq. 8 in the paper
    MM=sparse.csc_matrix(MM.T)
    Z=  X * MM
    Z= (Z == 2.0)
    Z=Z*1.0
    return [Z, CCmask]

### training-function of higher-order model
@jit
def train_higher(XtX, XtXdiag,lambdaBB, ZtZ, ZtZdiag, lambdaCC, CCmask, ZtX, rho, epochs):
    # precompute for BB
    ii_diag=np.diag_indices(XtX.shape[0])
    XtX[ii_diag] = XtXdiag+lambdaBB
    PP=np.linalg.inv(XtX)
    
    # precompute for CC
    ii_diag_ZZ=np.diag_indices(ZtZ.shape[0])
    ZtZ[ii_diag_ZZ] = ZtZdiag+lambdaCC+rho
    
    # initialize
    CC = np.zeros((ZtZ.shape[0], XtX.shape[0]),dtype=np.float64)
    DD = np.zeros((ZtZ.shape[0], XtX.shape[0]),dtype=np.float64)
    UU = np.zeros((ZtZ.shape[0], XtX.shape[0]),dtype=np.float64) # is Gamma in paper
    XtX[ii_diag] = XtXdiag
    
    for iter in range(epochs):
        print("epoch {}".format(iter))
        # learn BB
        #BB= PP.dot(XtX-ZtX.T.dot(CC)) 
        #gamma = np.diag(BB) / np.diag(PP)
        #BB -= PP * gamma

        BB= PP.dot(ZtX.T.dot(CC)) 
        gamma = (1 - np.diag(BB)) / np.diag(PP)
        BB = np.eye(n_items) - BB + PP * gamma
        
        # learn CC
        CC = np.linalg.solve(ZtZ, ZtX-ZtX.dot(BB) + rho * (DD-UU))
        
        # learn DD
        DD =  CC  * CCmask 
        #DD= np.maximum(0.0, DD) # if you want to enforce non-negative parameters
        
        # learn UU (is Gamma in paper)
        UU += CC-DD
        print(BB.max(), BB.min(), CC.max(), CC.min())
    return [BB,CC,DD]

#### data formation 
def format_data(tp):
    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [9]:
### choose the training-hyperparameters
epochs = 40 
threshold, lambdaBB, lambdaCC, rho = 4, 500, 5000, 100000

In [10]:
### format data ###
X = format_data(train_data)
del train_data

In [11]:
XtX=np.array((X.transpose() * X).todense()) 
XtXdiag=deepcopy(np.diag(XtX))

### create the list of feature-pairs and the higher-order matrix Z
XtX[np.diag_indices(XtX.shape[0])]=XtXdiag #if code is re-run, ensure that the diagonal is correct
ii_feature_pairs = create_list_feature_pairs(XtX, threshold)
print("number of feature-pairs: {}".format(len(ii_feature_pairs[0])))
Z, CCmask = create_matrix_Z(ii_feature_pairs, X)

### create the higher-order matrices
ZtZ=np.array((Z.transpose() * Z).todense()) 
ZtX=np.array((Z.transpose() * X).todense()) 
ZtZdiag=deepcopy(np.diag(ZtZ))
del X, Z

number of feature-pairs: 4171


In [12]:
### iterative training, and evaluation every 10 epochs 
BB, CC, DD = train_higher(XtX, XtXdiag, lambdaBB, ZtZ, ZtZdiag, lambdaCC, CCmask, ZtX, rho, epochs)

epoch 0
2.0 -0.14849171077822657 2.840962807823277e-05 -0.0007626453099606382
epoch 1
2.0043173531975644 -0.14849463428997553 5.524418650589409e-05 -0.00035378179323302577
epoch 2
2.0006279514553076 -0.14849449333889267 8.061489445696383e-05 -0.0005169510629481182
epoch 3
2.000808123081036 -0.14849567526407162 0.00010460119853334152 -0.0006714564010133423
epoch 4
2.0010470596728087 -0.14849686033756843 0.00012727885836483026 -0.0008177726647850774
epoch 5
2.0012741089046777 -0.14849798817985174 0.00014871950760823315 -0.0009563367332311784
epoch 6
2.0014886743100115 -0.14849905885977843 0.00016899084764837748 -0.0010875616381246281
epoch 7
2.0016914175663367 -0.1485000751516516 0.00018815686264271147 -0.0012118382794570454
epoch 8
2.001882991326013 -0.14850103981628054 0.00020627802418807012 -0.001329536626657571
epoch 9
2.0020640122982045 -0.14850195547984754 0.00022341148475130666 -0.0014410068449425399
epoch 10
2.0022350630552186 -0.14850282463502612 0.0002396112604192237 -0.0015465

In [13]:
del ZtZ, ZtX, ZtZdiag

# prediction

In [14]:
### evaluation-function of higher-order model
@jit
def pred_func(BB,CC,test_data_tr, Z_test_data_tr, N_test, batch_size_test=5000):
    print("Evaluating ...")
    #evaluate in batches
    for bnum, st_idx in tqdm(enumerate(range(0, N_test, batch_size_test))):
        end_idx = min(st_idx + batch_size_test, N_test)
        Xtest = test_data_tr[st_idx:end_idx]
        Ztest = Z_test_data_tr[st_idx:end_idx]
        if sparse.isspmatrix(Xtest):
            Xtest = Xtest.toarray()
            Ztest = Ztest.toarray()
        Xtest = Xtest.astype('float64')
        Ztest = Ztest.astype('float64')
        pred_val = (Xtest).dot(BB) + Ztest.dot(CC)
        #pred_val[Xtest.nonzero()] = -np.inf # exclude examples from training and validation (if any)
        pred_val = np.argpartition(pred_val, -12)[:,-12:]
        if bnum == 0:
            ans = pred_val
        else:
            ans = np.concatenate((ans,pred_val), axis=0)
    return ans

In [15]:
df = pd.read_csv(DATA_DIR + "transactions_train.csv")
df = df[["t_dat", "customer_id", "article_id"]]

valid_data = df.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
valid_unq.drop("article_id", axis=1, inplace=True)

valid_data = numerize(valid_data)
print(valid_data.shape)
X_valid = format_data(valid_data)
del valid_data, df

Z_val_data , _ = create_matrix_Z(ii_feature_pairs, X_valid)
N_valid = X_valid.shape[0]

(64996, 2)


In [16]:
id2show = dict((v,"0"+str(k)) for k,v in show2id.items())
id2profile = dict((v,k) for k,v in profile2id.items())

sample_value = sub_df["prediction"][0]

In [17]:
pred = pred_func(BB,CC, X_valid, Z_val_data, N_valid)

Evaluating ...


11it [08:45, 47.74s/it]


In [18]:
# https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key
# indexから商品idに変換
pred = np.vectorize(id2show.get)(pred)
pred = pd.DataFrame(pred)

In [19]:
cols = pred.columns
pred['prediction'] = pred[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
pred = pred.drop(cols, axis=1).reset_index(drop=False)
pred.columns = ["customer_id", "prediction"]
pred["customer_id"] = pred["customer_id"].map(id2profile)

In [20]:
sub_df = pd.merge(sub_df["customer_id"], pred, on = "customer_id", how="left")
sub_df["prediction"].fillna(sample_value, inplace=True)
sub_df.to_csv("submission.csv", index=False)

In [21]:
# local score
valid_unq = pd.merge(valid_unq, pred, on ="customer_id", how="left")
mapk(valid_unq['valid_true'].map(lambda x: x.split()), valid_unq['prediction'].map(lambda x: x.split()), k=12)

0.2134497415770507

# check

In [22]:
sub_df["prediction"].value_counts(), sub_df.shape

(0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001    1320957
 0882651001 0634541002 0640001033 0759482007 0883689001 0824752001 0910199002 0730454060 0877369002 0730863012 0875272001 0864339003      30790
 0918292001 0882902001 0730683062 0902802005 0740519002 0673677002 0788575002 0871517014 0874110016 0573085057 0714790020 0804992014         47
 0762796013 0884319005 0814686004 0873279005 0887757001 0924243001 0898694002 0849588008 0793012001 0831431001 0733803001 0928351001         35
 0923340001 0873274002 0909721001 0751471001 0903992003 0714790020 0928210002 0863583001 0896169002 0918522001 0918890002 0894780001         35
                                                                                                                                         ...   
 0884319005 0929275001 0573085057 0867969003 0673677002 0842000003 0906226001 0909721002 0706016001 0924243002 0894780001 0762846006    

In [23]:
valid_unq["prediction"].value_counts(), valid_unq.shape

(0918292001 0882902001 0730683062 0902802005 0740519002 0673677002 0788575002 0871517014 0874110016 0573085057 0714790020 0804992014    47
 0762796013 0884319005 0814686004 0873279005 0887757001 0924243001 0898694002 0849588008 0793012001 0831431001 0733803001 0928351001    35
 0923340001 0873274002 0909721001 0751471001 0903992003 0714790020 0928210002 0863583001 0896169002 0918522001 0918890002 0894780001    35
 0889550002 0924243002 0935541001 0929165002 0909721002 0909370001 0933032002 0918890002 0762796013 0895610005 0923758001 0896169005    31
 0919273001 0934835001 0902388001 0915453005 0909093003 0933032001 0673677002 0785034009 0714790020 0898694001 0924243002 0849588008    29
                                                                                                                                        ..
 0850917001 0915529003 0751471001 0924243001 0749974009 0867729001 0372860002 0831861004 0919106001 0814631001 0832307007 0570004010     1
 0573085057 0919365008 0762